In [1]:
import findspark
findspark.init('/home/osboxes/spark-2.4.3-bin-hadoop2.7')
import pyspark
import os

In [2]:
myPath = os.path.join('/home', 'osboxes', 'CourseMaterial', 'Spark_for_Machine_Learning', 'Logistic_Regression')

print(myPath)

os.chdir(myPath)

/home/osboxes/CourseMaterial/Spark_for_Machine_Learning/Logistic_Regression


In [3]:
os.listdir()

['Titanic_Log_Regression_Code_Along.ipynb',
 'new_customers.csv',
 '.ipynb_checkpoints',
 'titanic.csv',
 'sample_libsvm_data.txt',
 'Logistic_Regression_Consulting_Project_SOLUTIONS.ipynb',
 'Logistic_Regression_Example.ipynb',
 'Logistic_Regression_Consulting_Project.ipynb',
 'customer_churn.csv']

In [4]:
# set up the session info

from pyspark.sql import SparkSession

name = 'titanic'

spark = SparkSession.builder.appName(name).getOrCreate()

from pyspark.ml.classification import LogisticRegression

In [5]:
# get the titanic data

df = spark.read.csv('titanic.csv', inferSchema=True, header=True)

In [6]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [7]:
df.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [8]:
my_cols = df.select(['Survived',
                    'Pclass',
                    'Sex',
                    'Age',
                    'SibSp',
                    'Parch',
                    'Fare',
                    'Embarked'])

In [9]:
my_cols.show()

+--------+------+------+----+-----+-----+-------+--------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|
+--------+------+------+----+-----+-----+-------+--------+
|       0|     3|  male|22.0|    1|    0|   7.25|       S|
|       1|     1|female|38.0|    1|    0|71.2833|       C|
|       1|     3|female|26.0|    0|    0|  7.925|       S|
|       1|     1|female|35.0|    1|    0|   53.1|       S|
|       0|     3|  male|35.0|    0|    0|   8.05|       S|
|       0|     3|  male|null|    0|    0| 8.4583|       Q|
|       0|     1|  male|54.0|    0|    0|51.8625|       S|
|       0|     3|  male| 2.0|    3|    1| 21.075|       S|
|       1|     3|female|27.0|    0|    2|11.1333|       S|
|       1|     2|female|14.0|    1|    0|30.0708|       C|
|       1|     3|female| 4.0|    1|    1|   16.7|       S|
|       1|     1|female|58.0|    0|    0|  26.55|       S|
|       0|     3|  male|20.0|    0|    0|   8.05|       S|
|       0|     3|  male|39.0|    1|    5| 31.275|       

In [10]:
# for simplicity remove the na

final_data = my_cols.na.drop()

In [11]:
# get libraries for handling categoricals
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoder,StringIndexer)

In [12]:
# first do the gender one
# step 1 is to first assign a number index to the strings
# step 2 creates the dummy variables using one hot encoding
gender_indexer = StringIndexer(inputCol='Sex',outputCol='SexIndex')
gender_encoder = OneHotEncoder(inputCol='SexIndex',outputCol='SexVec')

In [13]:
# then embark
embark_indexer = StringIndexer(inputCol='Embarked',outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex',outputCol='EmbarkVec')

In [14]:
# create the assembler to convert them to features

assembler = VectorAssembler(inputCols=['Pclass',
                                       'SexVec',
                                       'Age',
                                       'SibSp',
                                       'Parch',
                                       'Fare',
                                       'EmbarkVec'],outputCol='features')

In [15]:
# get the model
from pyspark.ml.classification import LogisticRegression

In [16]:
# use the pipeline feature

from pyspark.ml import Pipeline

In [17]:
# create the logistic regression object

log_reg_titanic = LogisticRegression(featuresCol='features',labelCol='Survived')

In [18]:
# create the pipeline
# executes the steps that were created above

pipeline = Pipeline(stages=[gender_indexer,embark_indexer,
                           gender_encoder,embark_encoder,
                           assembler,log_reg_titanic])

In [19]:
train, test = final_data.randomSplit([0.7, 0.3])

In [20]:
fit_model = pipeline.fit(train)

In [22]:
# test the results

results = fit_model.transform(test)

In [23]:
# get an evaluator

from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [24]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Survived')

In [25]:
results.select('Survived', 'prediction').show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
+--------+----------+
only showing top 20 rows



In [27]:
AUC = my_eval.evaluate(results)
AUC

0.81616568914956